# Underdog Best Ball Mania III Analysis
- The goal of this analysis is to analyze the pick-by-pick data and results of the tournament to identify trends and/or insights of what strategies were successful (or not), which players over/under-performed vs expectation, and any other fun tidbits from the best Best Ball platform out there

- The purpose of this notebook is to
    - load the data from the CSV files downloaded from this Underdog Network link into Google BigQuery tables
        https://underdognetwork.com/football/best-ball-research/best-ball-mania-iii-downloadable-pick-by-pick-data
    - Aggregate the individual files/tables into 1 master dataset
    - Create additional features in that table to be used for analysis

- The data prepared from this process will be used to create an interactive Tableau dashboard that can be viewed via the link below (this is work-in-progress, more updates to come in the dashboard)
    https://public.tableau.com/app/profile/matt8622/viz/BestBallMania2022Analysis

# -------------------------------------------------------------------------------------------------------------

**Prep libraries and directories**

In [1]:
import google.cloud.bigquery as bq
import numpy as np
import pandas as pd
import pandas_gbq as pd_gbq
import time
from os import listdir
from os.path import join, isfile

pd.set_option('display.max_columns', 50)

# client = bq.Client.from_service_account_json('C:\\Users\\mshaw\\Desktop\\lively-paratext-319616-91376bb98d65.json')

# ## Custom function for runtime
# def runtime(start):
#     print('Runtime: ' + str(np.round((time.time() - start)/60, 1)) + ' minutes')    

WD = 'C:\\Users\\Matt\\Desktop\\F.F\\BB Data\\'   

## Project ID for FF Analysis
proj_id = 'ff-analysis'

# Data Guide

**https://underdognetwork.com/football/best-ball-research/best-ball-mania-iii-downloadable-pick-by-pick-data**

<font color = grey>

draft_id (the draft room) 

draft_time (date and time of day) 

clock (fast vs. slow drafts) 

draft_entry_id (can ignore) 

tournament_entry_id (draft team id)
tournament_round_draft_entry_id (also can ignore)

tournament_round_number (1 is the "regular season", 2 is second round, 3 is semifinals, 4 is finals) 

player_name (I hope you can figure that one out)

position_name, 

bye_week, 

projection_adp (just average draft position) 

pick_order (the team's draft placement) 

overall_pick_number,

team_pick_number (round)

pick_points (the amount of points this pick attributed to the best ball team in that round) 

roster_points (the team's total points in that round) 

playoff_team (only let's you know if it's in the playoff round).

# Load data into GBQ

**Regular Season**

In [14]:
## Substring of the files to be added
file_name = 'BBM_III_Regular_Season'

## Save list of all file names to import
filenames = [f for f in listdir(WD) if isfile(join(WD, f)) and file_name in f]

## Create dictionary shell
df_reg = pd.DataFrame(columns = pd.read_csv(WD + filenames[0]).columns)

## Add each files's data as a dataframe in the dictionary
for f in filenames:
    
    ## Read file
    temp = pd.read_csv(WD + f) 
    
    ## Add current file's data to overall dataframe
    df_reg = pd.concat([df_reg, temp], axis = 0)

df_reg.drop_duplicates(inplace = True)    
    
print(df_reg.shape)    

df_reg.head()

(16243200, 17)


,draft_id,draft_time,clock,draft_entry_id,tournament_entry_id,tournament_round_draft_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team
0,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,8d6dd37f-6a5d-420b-bc9c-b14f77588341,c2fd14f4-cfb8-405e-821d-6f577ce5be45,69c11d7e-b0c4-4771-b4a1-2e0cdce43da2,1,DJ Moore,WR,13,33.01,7,31,3,104.5,1478.34,0
1,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,372c3de1-5233-492f-ab1d-d1455630b9b5,b8f9e652-2362-4df9-b233-acee84698c9f,497c2ba9-4e49-47e3-b0c4-da045e76c2f7,1,James Cook,RB,7,102.69,9,105,9,35.5,1720.66,0
2,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,372c3de1-5233-492f-ab1d-d1455630b9b5,b8f9e652-2362-4df9-b233-acee84698c9f,497c2ba9-4e49-47e3-b0c4-da045e76c2f7,1,Malik Willis,QB,6,214.44,9,208,18,0.0,1720.66,0
3,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,a7c15a5c-8925-422c-979b-7c60fd0326c5,364bfa02-93db-417b-8c46-793759ef06d2,abd6356b-d301-4498-828f-f36c6e42af8c,1,Michael Carter,RB,10,140.28,2,146,13,74.1,1579.04,0
4,00012b23-0f2c-4727-b1ab-05b4a0120a76,2022-08-19 23:49:44.511157,14400,8d6dd37f-6a5d-420b-bc9c-b14f77588341,c2fd14f4-cfb8-405e-821d-6f577ce5be45,69c11d7e-b0c4-4771-b4a1-2e0cdce43da2,1,Darren Waller,TE,6,50.96,7,42,4,23.9,1478.34,0


In [16]:
## Assign data types to each column
df_reg = df_reg.astype({'draft_id':'string',
                        'draft_time':'string',
                        'clock':'int',
                        'draft_entry_id':'string',
                        'tournament_entry_id':'string',
                        'tournament_round_draft_entry_id':'string',
                        'tournament_round_number':'int',
                        'player_name':'string',
                        'position_name':'string',
                        'bye_week':'int',
                        'projection_adp':'float',
                        'pick_order':'int',
                        'overall_pick_number':'int',
                        'team_pick_number':'int',
                        'pick_points':'float',
                        'roster_points':'float',
                        'playoff_team':'int'})

## Write data to table in GBQ 
pd_gbq.to_gbq(df_reg, 'Source_Tables.BB_RegSeason', project_id = proj_id, if_exists = 'replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


**Quarter Finals**

In [18]:
## Substring of the files to be added
file_name = 'BBM_III_Quarterfinals'

## Save list of all file names to import
filenames = [f for f in listdir(WD) if isfile(join(WD, f)) and file_name in f]

## Create dictionary shell
df_qf = pd.DataFrame(columns = pd.read_csv(WD + filenames[0]).columns)

## Add each files's data as a dataframe in the dictionary
for f in filenames:
    
    ## Read file
    temp = pd.read_csv(WD + f) 
    
    ## Add file's data to overall dataframe
    df_qf = pd.concat([df_qf, temp], axis = 0)

df_qf.drop_duplicates(inplace = True)

print(df_qf.shape)    

df_qf.head()

(1353600, 17)


,draft_id,draft_time,clock,draft_entry_id,tournament_entry_id,tournament_round_draft_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team
0,00066e88-9401-4a03-a3bb-9ebd772e51d1,NaN,30,8928124e-f386-4760-925b-b4cb6965aeeb,4ecbcfef-bcd2-4d58-922e-83f049897ffe,393b32a0-f016-4d80-9174-470f74b65202,2,Michael Gallup,WR,9,0.0,0,11,11,0.0,123.14,1
1,00066e88-9401-4a03-a3bb-9ebd772e51d1,NaN,30,228adebd-cd52-4659-a184-91e344c830a5,4a3d4c25-9d62-407c-8e5b-fe342bf98e5d,9e896b68-9bd9-400b-87fd-8eba9b454d22,2,Van Jefferson,WR,7,0.0,0,140,14,0.0,99.44,1
2,00066e88-9401-4a03-a3bb-9ebd772e51d1,NaN,30,82afda48-91e2-4fa6-aeee-4b2fd15175d0,a2bae341-3cd4-476e-99d8-222c811ec269,df83f6b7-4fde-469f-809b-b32ded66e91d,2,Marlon Mack,RB,9,0.0,0,159,15,14.3,150.1,1
3,00066e88-9401-4a03-a3bb-9ebd772e51d1,NaN,30,84857aa4-d2ca-4fc5-921e-21f28b4b3baf,58ebcca2-bedb-403e-a70d-8b138fa9232d,705d7131-5301-47b4-9cab-2045bf83d59c,2,Cole Kmet,TE,14,0.0,0,100,10,4.5,107.3,1
4,00066e88-9401-4a03-a3bb-9ebd772e51d1,NaN,30,e6e813ae-17ad-4f5c-9829-2a06e0ea7283,4061a521-75e0-4094-9c8e-df6e58b46cfd,d5900340-915d-453b-a682-5e255f3f2adf,2,Trevor Lawrence,QB,11,0.0,0,121,13,27.82,140.32,1


In [7]:
## Assign data types to each column
df_qf = df_qf.astype({'draft_id':'string',
                        'draft_time':'string',
                        'clock':'int',
                        'draft_entry_id':'string',
                        'tournament_entry_id':'string',
                        'tournament_round_draft_entry_id':'string',
                        'tournament_round_number':'int',
                        'player_name':'string',
                        'position_name':'string',
                        'bye_week':'int',
                        'projection_adp':'float',
                        'pick_order':'int',
                        'overall_pick_number':'int',
                        'team_pick_number':'int',
                        'pick_points':'float',
                        'roster_points':'float',
                        'playoff_team':'int'})

# ## Project ID for FF Analysis
# proj_id = 'ff-analysis'

## Write data to table in GBQ 
pd_gbq.to_gbq(df_qf, 'Source_Tables.BB_Quarters', project_id = proj_id, if_exists = 'replace')

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 841.05it/s]


**Semi Finals**

In [ ]:
## Substring of the files to be added
file_name = 'Semifinals'

## Save list of all file names to import
filenames = [f for f in listdir(WD) if isfile(join(WD, f)) and file_name in f]

## Import data file
df_semi = pd.read_csv(WD + f) 

## Assign data types
df_semi = df_semi.astype({'draft_id':'string',
                        'draft_time':'string',
                        'clock':'int',
                        'draft_entry_id':'string',
                        'tournament_entry_id':'string',
                        'tournament_round_draft_entry_id':'string',
                        'tournament_round_number':'int',
                        'player_name':'string',
                        'position_name':'string',
                        'bye_week':'int',
                        'projection_adp':'float',
                        'pick_order':'int',
                        'overall_pick_number':'int',
                        'team_pick_number':'int',
                        'pick_points':'float',
                        'roster_points':'float',
                        'playoff_team':'int'})

## Project ID for FF Analysis
proj_id = 'ff-analysis'

## Write data to table in GBQ 
pd_gbq.to_gbq(df_semi, 'Source_Tables.BB_SemiFinals', project_id = proj_id, if_exists = 'replace')

**Finals**

In [ ]:
## Substring of the files to be added
file_name = 'Finals'

## Save list of all file names to import
filenames = [f for f in listdir(WD) if isfile(join(WD, f)) and file_name in f]

## Import data file
df_final = pd.read_csv(WD + f) 

## Assign data types
df_final = df_final.astype({'draft_id':'string',
                            'draft_time':'string',
                            'clock':'int',
                            'draft_entry_id':'string',
                            'tournament_entry_id':'string',
                            'tournament_round_draft_entry_id':'string',
                            'tournament_round_number':'int',
                            'player_name':'string',
                            'position_name':'string',
                            'bye_week':'int',
                            'projection_adp':'float',
                            'pick_order':'int',
                            'overall_pick_number':'int',
                            'team_pick_number':'int',
                            'pick_points':'float',
                            'roster_points':'float',
                            'playoff_team':'int'})

## Project ID for FF Analysis
proj_id = 'ff-analysis'

## Write data to table in GBQ 
pd_gbq.to_gbq(df_final, 'Source_Tables.BB_Finals', project_id = proj_id, if_exists = 'replace')

## -----------------------------------------------------------------------------------------------------------------------------

# Feature Engineering

Create the main data set for analysis

In [14]:
## Read from new table written to GBQ
sql = """

-- Quaterfinals data
WITH QUARTERS AS
(
  SELECT 
    tournament_entry_id,
    playoff_team,
    player_name,
    position_name,
    pick_points AS pick_points_quarters,
    roster_points AS roster_points_quarters
  FROM `Source_Tables.BB_Quarters` 
),

-- Semi-finals data
SEMIS AS 
(
  SELECT 
    tournament_entry_id,
    playoff_team,
    player_name,
    position_name,
    pick_points AS pick_points_semis,
    roster_points AS roster_points_semis
  FROM `Source_Tables.BB_SemiFinals` 
),

-- Finals data
FINALS AS 
(
  SELECT 
    tournament_entry_id,
    playoff_team,
    player_name,
    position_name,
    pick_points AS pick_points_finals,
    roster_points AS roster_points_finals
  FROM `Source_Tables.BB_Finals` 
),

-- Add all playoff data to regular season.  Add additional features.
BASE AS
(
    SELECT 
      A.*,

      -- Quarterfinal data
      CASE WHEN QF.tournament_entry_id IS NOT NULL THEN 1 ELSE 0
            END AS advance_quaters,
      pick_points_quarters,
      roster_points_quarters,

      -- Semifinal data  
      CASE WHEN SF.tournament_entry_id IS NOT NULL THEN 1 ELSE 0
            END AS advance_semis,
      pick_points_semis,
      roster_points_semis,

      -- Finals data  
      CASE WHEN F.tournament_entry_id IS NOT NULL THEN 1 ELSE 0
            END AS advance_finals,
      pick_points_finals,
      roster_points_finals,


      -- # of players per position picked through each round
      SUM(CASE WHEN A.position_name = 'QB' THEN 1 ELSE 0 END) 
        OVER(PARTITION BY A.tournament_entry_id ORDER BY A.team_pick_number) AS cumul_pos_qb,
      SUM(CASE WHEN A.position_name = 'RB' THEN 1 ELSE 0 END) 
        OVER(PARTITION BY A.tournament_entry_id ORDER BY A.team_pick_number) AS cumul_pos_rb,
      SUM(CASE WHEN A.position_name = 'WR' THEN 1 ELSE 0 END) 
        OVER(PARTITION BY A.tournament_entry_id ORDER BY A.team_pick_number) AS cumul_pos_wr,
      SUM(CASE WHEN A.position_name = 'TE' THEN 1 ELSE 0 END) 
        OVER(PARTITION BY A.tournament_entry_id ORDER BY A.team_pick_number) AS cumul_pos_te  

    FROM `Source_Tables.BB_RegSeason` A
    LEFT JOIN QUARTERS QF
      ON A.tournament_entry_id = QF.tournament_entry_id AND 
         A.player_name = QF.player_name AND
         A.position_name = QF.position_name
    LEFT JOIN SEMIS SF
      ON A.tournament_entry_id = SF.tournament_entry_id AND
         A.player_name = SF.player_name AND
         A.position_name = SF.position_name
    LEFT JOIN FINALS F
      ON A.tournament_entry_id = F.tournament_entry_id AND 
         A.player_name = F.player_name AND
         A.position_name = F.position_name  

    ORDER BY A.tournament_entry_id, A.team_pick_number
    LIMIT 1000
),

-- Classify draft strategies
DRAFT_STRATEGIES AS 
(
    SELECT DISTINCT
      tournament_entry_id,
      COALESCE(draft_strategy_rb_wr, 'Mixed') AS draft_strategy_rb_wr,
      COALESCE(draft_strategy_qb_te, 'Mixed') AS draft_strategy_qb_te
    FROM 
       (SELECT 
          tournament_entry_id,
          team_pick_number,
          player_name,

          -- RB/WR Strategy
          CASE WHEN team_pick_number = 3 AND cumul_pos_rb = 3 THEN 'Robust RB'
               WHEN team_pick_number = 6 AND cumul_pos_rb = 0 THEN 'Zero RB'
               WHEN team_pick_number = 2 AND cumul_pos_rb = 1 AND
                    team_pick_number = 6 AND cumul_pos_rb = 1 THEN 'Hero RB'   
               END AS draft_strategy_rb_wr,

          -- QB/TE Strategy
          CASE WHEN team_pick_number = 5 AND cumul_pos_qb >= 1 THEN 'Early QB' 
               WHEN team_pick_number = 5 AND cumul_pos_te >= 1 THEN 'Early TE'
               WHEN team_pick_number = 5 AND cumul_pos_qb >= 1 AND cumul_pos_te >= 1 THEN 'Early QB & TE'
               END AS draft_strategy_qb_te
        FROM BASE) A
),

FINAL_DATA AS
(
    SELECT
        A.*,
        B.draft_strategy_rb_wr,
        B.draft_strategy_qb_te
    FROM BASE A
    LEFT JOIN DRAFT_STRATEGIES B
        ON A.tournament_entry_id = B.tournament_entry_id
)

"""

df = pd_gbq.read_gbq(sql, project_id = proj_id)

# df_adv_met.head(3)

df.head()

Downloading: 100%|█████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2641.72rows/s]


,draft_id,draft_time,clock,draft_entry_id,tournament_entry_id,tournament_round_draft_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team,advance_quaters,pick_points_quarters,roster_points_quarters,advance_semis,pick_points_semis,roster_points_semis,advance_finals,pick_points_finals,roster_points_finals,cumul_pos_qb,cumul_pos_rb,cumul_pos_wr,cumul_pos_te
0,9a44a544-3b49-4c62-9958-1a475306c893,2022-08-29 17:59:50.184526,30,c2a73416-8875-4e66-b19e-0ca700bf58d2,0000146a-e245-49c3-8a4e-8e739cfe1e46,e9eccb8b-99c3-4e0c-a6a3-076671ee6fc4,1,Austin Ekeler,RB,8,5.82,6,6,1,246.20,1578.74,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,1,0,0
1,9a44a544-3b49-4c62-9958-1a475306c893,2022-08-29 17:59:50.184526,30,c2a73416-8875-4e66-b19e-0ca700bf58d2,0000146a-e245-49c3-8a4e-8e739cfe1e46,e9eccb8b-99c3-4e0c-a6a3-076671ee6fc4,1,Javonte Williams,RB,9,20.66,6,19,2,14.30,1578.74,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,2,0,0
2,9a44a544-3b49-4c62-9958-1a475306c893,2022-08-29 17:59:50.184526,30,c2a73416-8875-4e66-b19e-0ca700bf58d2,0000146a-e245-49c3-8a4e-8e739cfe1e46,e9eccb8b-99c3-4e0c-a6a3-076671ee6fc4,1,Josh Allen,QB,7,31.61,6,30,3,296.42,1578.74,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,2,0,0
3,9a44a544-3b49-4c62-9958-1a475306c893,2022-08-29 17:59:50.184526,30,c2a73416-8875-4e66-b19e-0ca700bf58d2,0000146a-e245-49c3-8a4e-8e739cfe1e46,e9eccb8b-99c3-4e0c-a6a3-076671ee6fc4,1,Gabe Davis,WR,7,40.91,6,43,4,107.30,1578.74,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,2,1,0
4,9a44a544-3b49-4c62-9958-1a475306c893,2022-08-29 17:59:50.184526,30,c2a73416-8875-4e66-b19e-0ca700bf58d2,0000146a-e245-49c3-8a4e-8e739cfe1e46,e9eccb8b-99c3-4e0c-a6a3-076671ee6fc4,1,Diontae Johnson,WR,9,48.81,6,54,5,102.50,1578.74,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,2,2,0


**Load FINAL table.  Export into CSV to be used in Tableau**

In [3]:
## Read from new table written to GBQ
sql = """

SELECT *
FROM `Source_Tables.BB_AGG` 

"""

df_final = pd_gbq.read_gbq(sql, project_id = proj_id)


df_final.to_csv('C:\\Users\\Matt\\Desktop\\BB\\BBM22_AggData.csv',index = False)

Downloading: 100%|███████████████████████████████████████████████████████| 8542026/8542026 [55:46<00:00, 2552.45rows/s]


,draft_id,draft_time,clock,draft_entry_id,tournament_entry_id,tournament_round_draft_entry_id,tournament_round_number,player_name,position_name,bye_week,projection_adp,pick_order,overall_pick_number,team_pick_number,pick_points,roster_points,playoff_team,advance_quaters,pick_points_quarters,roster_points_quarters,advance_semis,pick_points_semis,roster_points_semis,advance_finals,pick_points_finals,roster_points_finals,cumul_pos_qb,cumul_pos_rb,cumul_pos_wr,cumul_pos_te,draft_strategy_rb_wr,draft_strategy_qb_te,picks_per_pos,row_n
0,92ebdec8-f101-43d7-bb27-5ac39a16908d,2022-07-21 08:29:03.719297,30,716b9697-e1c9-47a8-b5a1-4a23fe6a7aca,8e5fadf8-a788-4893-b196-43b2c23f769f,25934c9e-7cc1-4680-a4ba-e86c15b39b39,1,Russell Wilson,QB,9,76.76,11,83,7,107.92,1459.20,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,5,1,0,Robust RB,Mixed,"2, 7, 6, 3",3232218
1,c056c975-c394-42fe-b6b5-0e989af41123,2022-07-19 04:42:24.772322,30,3a00b7fb-4293-4468-8b31-78336897c5c1,8e65559b-56f0-4c50-88c3-d11fc2e7707a,5278aef1-d696-4e03-8a51-5b9b6bbaa414,1,Zach Wilson,QB,10,175.21,3,166,14,20.28,1328.10,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,2,4,6,2,Mixed,Early QB,"3, 6, 7, 2",7692112
2,ff8128b5-2713-4b40-85d3-a8e66d4a6d32,2022-07-07 02:57:06.820950,30,76e3185c-dddc-470f-9e65-b08f43743f37,8e6720fc-2f1f-474f-8375-1a300c471a25,ee5d8d1f-4f92-4808-a79d-f85d70c5a3a9,1,Robert Woods,WR,6,102.43,5,116,10,39.90,1565.04,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,4,4,1,Mixed,Mixed,"2, 6, 7, 3",5341107
3,b7eb9b75-aa9b-42b6-a97f-506fb91677d0,2022-06-20 19:00:04.366217,30,7eef6347-764f-41fc-87b6-738f4f416231,8e6c17ec-4925-4222-8bd3-88e600b0e8a2,eb47f836-1668-4252-a9f9-6a33a65f3d04,1,Curtis Samuel,WR,14,176.83,12,157,14,111.30,1669.60,0,1,0.0,140.94,1,13.9,94.76,0,NaN,NaN,1,4,6,3,Mixed,Early QB,"2, 5, 8, 3",6021021
4,3b6e71fd-7947-49cd-baa8-8abb93735eca,2022-08-04 15:31:58.653496,28800,557a86ed-44a3-4d0b-adbe-b24fafa18910,8e6ea10d-a74a-4bf7-9d75-d174130964af,73af6319-f10c-416d-a3e3-03c439373ffd,1,Robert Woods,WR,6,101.15,3,99,9,50.40,1321.76,0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,1,3,4,1,Mixed,Early QB,"2, 5, 9, 2",8443470
